In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.fftpack as fp
import collections
import cv2
from __future__ import print_function
from pprint import pprint
import math
from numpy import binary_repr
from skimage import color, data, restoration
from scipy.fftpack import fftn, ifftn, fftshift
from skimage import img_as_float

# Question 1

<b>Describe, in your own words, what are sampling and quantization. Given the function $f(x) = -0.5x^2 + 3.5x + 1$, generate the binary code that represents the digitized signal using a sampling rate of 0.5 units in the interval [0, 5] and 16 grey levels. Consider that the grey levels 0 and 15 are equal to the function values 0 and 7.5, respectively.</b>

Sampling and quantization are related to the capture of images through sensors like cameras. The sampling is the mapping of coordinate and quantization is the attribution of value of that coordinate.

For example, we gonna start the capture of one image through a single sensor that identify one color in grayscale. We could define that image will be 8x8 shape. In this case, the sensor will verify the first position, this is the *sampling* procedure, the sensor is mapping or digitalizing that position [0,0] will have a value. The sampling always has to be equally spaced between the samples. Then, we perform the *quantization*, where the sensor will generate a signal related to the value that will be storage to that coordinate. The signal can vary in the format. In this example the signal can be from 0 to 1024, where we get 400 at this position, so it's required scale this value to 0-255 scale, in this case we gonna storage the 99 value at [0,0] coordinate. This procedure is repeated for all coordinates.

The function **exFunc** from next code is the implementation of the equation $f(x) = -0.5x^2 + 3.5x + 1$, the for instruction call the function in the interval of x = [0,5]. The function **rescale15** take the result and rescale in 16 levels of grayscale. Then, the binary code is printed along with a chart representing the values.


In [ ]:
def exFunc(x):
    return -0.5*(x**2) + 3.5 * x + 1

def rescale15(v):
    return int(round(v*15/7.5))

arr = []
print("Signal\tScale\tBin")
for v in np.arange(0,5,0.5):
    nV = exFunc(v)
    res = rescale15(nV)
    arr.append(res)
    print(nV,"\t", res, "\t",binary_repr(res,4))
    
plt.title("Signal in scale 0-15")
plt.plot(arr)
plt.show()

# Question 2
<b>Implement, from scratch, convolution on the spatial domain. Your function should receive the kernel and its dimensions as parameters. Considering that the kernel is square and has odd dimensions d, ignore the first and last d=2 rows and columns. Use it to perform low-pass and high-pass filtering on some images.</b>

To solve this question I created the function **spatialFilter** that basically receive the image, the kernel and the size of kernel to be apply on the image.

In [ ]:
def spatialFilter(img, kernel, kernel_size):    
    new_img = np.zeros_like(img)
    divided = math.floor(kernel_size/2)
    
    for i in range(divided,img.shape[0]-divided,1):
        for j in range(divided,img.shape[1]-divided,1):
            space = img[i-divided:i+divided+1,j-divided:j+divided+1]
            new_img[i,j] = np.sum(space*kernel)/ (kernel_size*kernel_size)
            
    return new_img
    

### Low Pass Filter

To illustrate the application of Low Pass Filter, I created the function **lowPassFilter** where only is needed to pass the image. Inside this function is defined one kernel with 15x15, and it will apply my filter and the OpenCV filter for comparison. So I decided to test in the mig.jpg image and in the image that the book of course  uses to realize these tests.

In [ ]:
def lowPassFilter(img):

    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 10, forward=True)
    ax[0].imshow(img, cmap=plt.cm.gray)
    ax[0].title.set_text("Original")

    kernel = np.ones((15,15))
    
    img2 = spatialFilter(img, kernel=kernel, kernel_size=kernel.shape[0])
    ax[1].imshow(img2, cmap=plt.cm.gray)
    ax[1].title.set_text("My blur")
    
    
    img2 = cv2.blur(img,(15,15),0)
    ax[2].imshow(img2, cmap=plt.cm.gray)
    ax[2].title.set_text("OpenCV blur")
    plt.show()
    

img = cv2.imread('mit.jpg', 0)
lowPassFilter(img)

img = cv2.imread('a.png', 0)
lowPassFilter(img)

### High Pass Filter

To illustrate the *High Pass Filter*, I created four different kernels to be applied through my function *spatialFilter* and the OpenCV version.

In [ ]:
def highPassFilter(img):   
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 15, forward=True)
    
    
    #kernel 1
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    
    #essa divisão pelo shape só pode ser feita na minha funcao, a funcao do OpenCv aparentemente já faz isso
    #e ela só serve no high pass, nao pode ser usada no low pass
    k = kernel/(kernel.shape[0]*kernel.shape[1])
    
    img2 = spatialFilter(img, kernel=k, kernel_size=kernel.shape[0])
    ax[0].imshow(img2, cmap=plt.cm.gray)
    ax[0].title.set_text("My high pass kernel 1")
    
    img2 = cv2.filter2D(img,-20,kernel)
    ax[1].imshow(img2, cmap=plt.cm.gray)
    ax[1].title.set_text("OpenCV HighPass")
    
    
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 15, forward=True)
    #kernel 2
    kernel = np.array([[1,1,1], 
                       [1,-8,1],
                       [1,1,1]])
    
    k = kernel/(kernel.shape[0]*kernel.shape[1])
    
    img2 = spatialFilter(img, kernel=k, kernel_size=kernel.shape[0])
    ax[0].imshow(img2, cmap=plt.cm.gray)
    ax[0].title.set_text("My high pass kernel 2")
    
    img2 = cv2.filter2D(img,-1,kernel)
    ax[1].imshow(img2, cmap=plt.cm.gray)
    ax[1].title.set_text("OpenCV HighPass")
    
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 15, forward=True)
    #kernel 3
    kernel = np.array([[0, -1, 0],
                       [-1, 4, -1],
                       [0, -1, 0]])    
    
    k = kernel/(kernel.shape[0]*kernel.shape[1])
    
    
    img2 = spatialFilter(img, kernel=k, kernel_size=kernel.shape[0])
    ax[0].imshow(img2, cmap=plt.cm.gray)
    ax[0].title.set_text("My high pass kernel 3")
    
    img2 = cv2.filter2D(img,-1,kernel)
    ax[1].imshow(img2, cmap=plt.cm.gray)
    ax[1].title.set_text("OpenCV HighPass")
    
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 15, forward=True)
    #kernel 4
    kernel = np.array([[-1,-1,-1], 
                       [-1,8,-1],
                       [-1,-1,-1]])
    
    k = kernel/(kernel.shape[0]*kernel.shape[1])
    
    img2 = spatialFilter(img, kernel=k, kernel_size=kernel.shape[0])
    ax[0].imshow(img2, cmap=plt.cm.gray)
    ax[0].title.set_text("My high pass kernel 4")
    
    img2 = cv2.filter2D(img,-1,kernel)
    ax[1].imshow(img2, cmap=plt.cm.gray)
    ax[1].title.set_text("OpenCV HighPass")
    
    
    plt.show()
    
    
img = cv2.imread('mit.jpg', 0)
highPassFilter(img)

img = cv2.imread('a.png', 0)
highPassFilter(img)

# Question 3

<b>Implement, from scratch, the histogram equalization technique and apply it to the images of the test set with low contrast.</b>

To solve this question I created four functions:

- **histTable** to generate the histogram in Dict format (Dict is the python hashtable)
- **getCdf** to calculate the *Cumulative distribution*
- **equalized** that can receive one image and call the previous functions to generate the equalization table or receive that personalized equalization table. (I decided to create this function in this way because I was doing others tests from the book)
- **equalize** finally the function to apply the equalization in one image.


In [ ]:
#Gera o histograma
#Uma lista com a distribuição de todos os tons presentes na imagem
#E a quantidade de vezes que a tonalidade está presente
def histTable(img):
    hist = {}
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            color = img[i,j]
            
            if color in hist.keys():
                hist[color] += 1
            else:
                hist[color] = 1
    #Tem que ser ordenado
    return collections.OrderedDict(sorted(hist.items()))

In [ ]:
#Cálculo da Cumulative distribuition
#Para cada tonaliade é somada a sua quantidade com a quantidade de tonalidades anteriores
#https://en.wikipedia.org/wiki/Histogram_equalization
def getCdf(img,hist=None):
    if hist == None:
        hist = histTable(img)
    
    cdf = {}
    for i in hist:
        cdf[i] = 0
        for j in hist:
            cdf[i] += hist[j]
            if j == i:
                break
    
    return cdf

In [ ]:
#Gera a tabela com os valores equalizados
#A chave da tabela é o valor atual da tonalidade e o valor é a nova tonalidade
def equalized(img,hist=None,space=256):
    cdf = getCdf(img,hist)    
    _minCdf = cdf[min(cdf)]
    
    eq = {}
    for i in cdf:
        try:
            r = round((cdf[i] - _minCdf)/ (img.shape[0]*img.shape[1] - _minCdf) * space-1)
        except:
            r = 0
        eq[i] = r
    return eq


In [ ]:
#Aplica a equalização nas imagens
def equalize(img,hist=None,space=256):
    new = np.zeros_like(img)
    eq = equalized(img,hist,space)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            color = img[i,j]
            
            if color in eq.keys():
                new[i,j] = eq[color]
    return new

In [ ]:
img = cv2.imread('Google_JAX_low_contrast.jpg', 0)
fig, ax = plt.subplots(2, 3)
fig.set_size_inches(15, 10, forward=True)
pos = [(j,i) for j in range(4) for i in range(3)]
p = 0

#A
ax[pos[p]].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=255)
p += 1

img2 = equalize(img)
ax[pos[p]].imshow(img2, cmap=plt.cm.gray, vmin=0, vmax=255)
p += 1

histr = cv2.calcHist([img2], [0], None, [256], [0, 256])
histr_orig = cv2.calcHist([img], [0], None, [256], [0, 256])
ax[pos[p]].plot(histr,label='equalized')
ax[pos[p]].plot(histr_orig,label='original')
ax[pos[p]].axis(xmin=0,xmax=256)
ax[pos[p]].legend(loc='upper right')
p += 1

#B
img = cv2.imread('woman_low_contrast.jpg', 0)
ax[pos[p]].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=255)
p += 1

img2 = equalize(img)
ax[pos[p]].imshow(img2, cmap=plt.cm.gray, vmin=0, vmax=255)
p += 1

histr = cv2.calcHist([img2], [0], None, [256], [0, 256])
histr_orig = cv2.calcHist([img], [0], None, [256], [0, 256])
ax[pos[p]].plot(histr,label='equalized')
ax[pos[p]].plot(histr_orig,label='original')
ax[pos[p]].axis(xmin=0,xmax=256)
ax[pos[p]].legend(loc='upper right')
p += 1

# Question 4
<b>Implement, from scratch, the median filter and apply it to the images with
salt-and-pepper noise.</b>

To solve this question I implemented the function *medianFilter* similar to *spatialFilter*, but in this one has the calculation of median in each image subarea. Then I applied this function on the images through the function *applyMedian*, that will call my function and the OpenCV function to comparison.

In [ ]:
def medianFilter(img, kernel_size):
        
    new_img = np.zeros_like(img)
    divided = math.floor(kernel_size/2)
    
    for i in range(divided,img.shape[0]-divided,1):
        for j in range(divided,img.shape[1]-divided,1):
            space = img[i-divided:i+divided+1,j-divided:j+divided+1]
            new_img[i,j] = np.median(space)
            
    return new_img

In [ ]:
def applyMedian(img, name=""):
    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 10)

    my_median = medianFilter(img,7)
    median = cv2.medianBlur(img,7)

    ax[0].imshow(img, cmap=plt.cm.gray)
    ax[0].title.set_text('Original %s' % name)
    
    ax[1].imshow(my_median, cmap=plt.cm.gray)
    ax[1].title.set_text('My median')
    
    ax[2].imshow(median, cmap=plt.cm.gray)
    ax[2].title.set_text('OpenCV Median')
    plt.show()
    

img = cv2.imread('bear_s_and_p.png', 0)
applyMedian(img,'bear_s_and_p')
img = cv2.imread('boat_s_and_p.png', 0)
applyMedian(img,'boat_s_and_p')
img = cv2.imread('lions_s_and_p.png', 0)
applyMedian(img,'lions_s_and_p')
img = cv2.imread('glass_s_and_p.png', 0)
applyMedian(img,'glass_s_and_p')
img = cv2.imread('lungs_s_and_p_10.jpg', 0)
applyMedian(img,'lungs_s_and_p_10')
img = cv2.imread('lungs_s_and_p_30.jpg', 0)
applyMedian(img,'lungs_s_and_p_30')
img = cv2.imread('lungs_s_and_p_50.jpg', 0)
applyMedian(img,'lungs_s_and_p_50')


## The book example
It is worth noting that, the book presents this example, where the author says that was applied the median filter on the image A, and he got the image D, but I could't perform the same effect, neither with my function nor with OpenCV.

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(10, 10)


img = cv2.imread('16-circuit-salt-and-pepper.png', 0)

my_median = medianFilter(img,7)
median = cv2.medianBlur(img,7)

ax[0,0].imshow(img, cmap=plt.cm.gray)
ax[0,0].title.set_text('Original A')

ax[0,1].imshow(my_median, cmap=plt.cm.gray)
ax[0,1].title.set_text('My median B')

ax[1,0].imshow(median, cmap=plt.cm.gray),
ax[1,0].title.set_text('OpenCV Median C')

img = cv2.imread('16-circuit-salt-and-pepper-median.png', 0)
ax[1,1].imshow(img, cmap=plt.cm.gray),
ax[1,1].title.set_text('Expectation D')


# Question 5
**Using a Fourier transform library, implement the high-frequency emphasis
filter defined by:**

$$g(x,y) = \mathcal{F}^{-1}{[k_1 + k_2 H_{HP} (u, v)]F(u, v)}$$

**where $k_1 \leq 0$ offsets the value the transfer function so as not to zero-out
the dc term, and $k_2 > 0$ controls the contribution of high frequencies.
Process the image ''full body PET - original.jpg'', varying the values of $k_1$
and $k_2$ and select the best values, in your opinion.**

To solve this question I believe that I made one implementation slightly different, cause I implemented the function **filterMask** that will generate the mask of types IDEAL, BUTTERWORTH or GAUSSIAN, I only need to pass the shape, type and if will be an LOW_PASS or HIGH_PASS mask.

In [ ]:
mask_types = ["IDEAL_MASK","BUTTERWORTH_MASK","GAUSSIAN_MASK"]
IDEAL_MASK = 0
BUTTERWORTH_MASK = 1
GAUSSIAN_MASK = 2
LOW_PASS = 0
HIGH_PASS = 1

#IMPLEMENTACAO CORRETA
#mask_type=IDEAL_MASK = 0
#mask_type=BUTTERWORTH_MASK = 1
#mode=LOW_PASS
#mode=HIGH_PASS
def filterMask(dim,D0=None,n=1,mask_type=IDEAL_MASK,mode=LOW_PASS):
    
    if D0 == None:
        D0 = math.floor(dim[0]*0.5)
        if mask_type == GAUSSIAN_MASK:
            D0 = 0.05
        
    mask = np.zeros(dim)
    
    
    for u in range(dim[0]):
        for v in range(dim[1]):
            #Esta implementação está errada!
            #Duv = math.sqrt(u**2+v**2)
            #Essa daqui foi dificil, em cada lugar ta de um jeito diferente
            Duv = (dim[0]/2.0 - u)**2 + (dim[1]/2.0 - v)**2
            
            if mask_type==IDEAL_MASK:
                if Duv > D0**2:
                    mask[u,v] = 0
                elif Duv <= D0**2:
                    mask[u,v] = 1
            elif mask_type == GAUSSIAN_MASK:
                mask[u,v] = math.exp( -Duv**2/2*D0**2 )
            else:
                mask[u,v] = (1 / (1 + ((math.sqrt(2)-1) * Duv/D0))**(2*n))
    if (mode==HIGH_PASS):
        ones = np.ones_like(mask)
        mask = ones - mask
        
    return mask

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(7, 5)
mask = filterMask((30,30),mode=HIGH_PASS)
ax[0].imshow(mask, cmap = 'gray')
ax[0].title.set_text("IDEAL")

mask = filterMask((30,30),mask_type=GAUSSIAN_MASK,mode=HIGH_PASS)
ax[1].imshow(mask, cmap = 'gray')
ax[1].title.set_text("GAUSSIAN")

mask = filterMask((30,30), mask_type=BUTTERWORTH_MASK,mode=HIGH_PASS)
ax[2].imshow(mask, cmap = 'gray')
ax[2].title.set_text("BUTTERWORTH_MASK")
plt.show()


Then I implemented the function that will apply the mask in many configurations on the image:
- **getMag** Fourier transform
- **inverse** Inverse Fourier transform
- **spectrum** Applies the log on the spectrum to improve the visualization
- **rescale255** Rescale the matrix values to 0-255
- **runHPTests** Applies many configurations of the selected mask

In [ ]:

def getMag(img):
    F1 = fp.fft2((img).astype(float))
    F2 = fp.fftshift(F1)
    return F2

def inverse(mag):
    return fp.ifft2(fp.ifftshift(mag)).real

def spectrum(mag):
    return (20*np.log10( 0.1 + mag)).astype(int)


def rescale255(arr):
    div = (arr.max() - arr.min())
    #print (div)
    if div == 0:
        div = 1
    return ((arr - arr.min()) * (1/div * 255)).astype('uint8')

In [ ]:
def runHPTests(img,D0,mask_type):
    
    mag = getMag(img)
    mag_mod = mag.copy()

    step = 0.40
    mask_orig = filterMask(mag_mod.shape,
                            D0=D0,
                           mask_type=mask_type,
                           mode=HIGH_PASS)

    mag_mod = mask_orig * mag_mod
    inv = inverse(mag_mod)


    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(18, 10)
    fig.suptitle("%s D0=%d" % (mask_types[mask_type],D0))
    ax[0].imshow( img, cmap='gray')
    ax[0].title.set_text("Original")
    ax[1].imshow( mask_orig, cmap='gray')
    ax[1].title.set_text("Mask")
    ax[2].imshow( spectrum(mag_mod), cmap='gray')
    ax[2].title.set_text("Spectrum")
    plt.show()
    
    
    iteracao = 0

    for k2 in np.arange(0.0, 1.0+step, step):
        for k1 in np.arange(0.0, 1.0+step, step):
            iteracao += 1
            
            fig, ax = plt.subplots(1, 4)
            fig.set_size_inches(18, 7)
            fig.suptitle("ITERATION %d %s D0=%d k1=%0.2f k2=%0.2f" % (iteracao,mask_types[mask_type],D0,k1,k2))

            mag_mod = mag.copy()
            mask = mask_orig.copy()
            mag_mod = (k1 + k2 * mask) * mag_mod
            inv = inverse(mag_mod)
            uni = img+inv

            
            ax[0].imshow( inv, cmap='gray')
            ax[0].axis('off')
            ax[0].title.set_text('Inverse')
            
            ax[1].imshow( uni, cmap='gray')
            ax[1].axis('off')
            ax[1].title.set_text('Inverse+Orig')

            inv = rescale255(np.around(inv))
            uni = rescale255(np.around(uni))

            
            ax[2].imshow( equalize(inv), cmap='gray',vmin=0,vmax=255)
            ax[2].title.set_text('Inverse Equalized')
            ax[2].axis('off')
            
            ax[3].imshow( inv, cmap='gray',vmin=0,vmax=170)
            ax[3].title.set_text('Inverse vmax=170')
            ax[3].axis('off')

            plt.show()
            


The first image is the original, the second is the mask that will be applied, and the third is the result of that mask on the spectrum. The tested parameters modify the mask and consequentily modifies the spectrum, but the difference in the spectrum is almost imperceptible, for this reason I decided not reprint the spectrum.


Then I show the used configurations, ex: IDEAL_MASK D0=5 k1=0.00 k2=0.00 with the results.

- **Inverse** is the inverse Fourier transform after applying the mask
- **Inverse+Orig** is the sum of inverse with the original image
- **Inverse equalized** I noticed that on the figure 4.59 on the 3° ed of the book, has the equalized version of inverse, so i decided show this same equalization, but I was not satisfied with the result.
- **Inverse vmax=150** As I was not satisfied with the result of equalization, I decided to show the inverse with the [0-150] scale option instead of [0-255]. This generated a better result.


Looking to Inverse vmax=170, on the ninth iteration, D0=5, k1=0 and k2=0.80, with IDEAL_MASK or thirteenth iteration, D0=300, k1=0 and k2=1.2, with BUTTERWORTH_MASK or GAUSSIAN_MASK, is possible to see more clearly eight tiny points in the brain, beyond the big white point in the brain and two big white points in the right lung. 

In [ ]:
mag = getMag(cv2.imread('pet.jpg',0))
mask = filterMask(mag.shape,D0=5,
                       mask_type=IDEAL_MASK,
                       mode=HIGH_PASS)
mask2 = filterMask(mag.shape,D0=300,
                       mask_type=BUTTERWORTH_MASK,
                       mode=HIGH_PASS)

res1 = inverse((0 + 0.8 * mask) * mag)
res2 = inverse((0 + 1.2 * mask2) * mag)

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 10)

circles = []
circles.append(plt.Circle((125, 48), 5, color='w', fill=False))
circles.append(plt.Circle((130, 50), 5, color='w', fill=False))
circles.append(plt.Circle((133, 45), 5, color='w', fill=False))
circles.append(plt.Circle((140, 35), 5, color='w', fill=False))
circles.append(plt.Circle((145, 40), 5, color='w', fill=False))
circles.append(plt.Circle((146, 50), 5, color='w', fill=False))
circles.append(plt.Circle((153, 55), 5, color='w', fill=False))
circles.append(plt.Circle((130, 180), 12, color='w', fill=False))
circles.append(plt.Circle((110, 165), 12, color='w', fill=False))

for c in circles:
    ax[0].add_patch(c)

ax[0].imshow(res1,cmap='gray',vmax=170)
ax[0].title.set_text("Inverse vmax=170 9° ITER D0=5 k1=1.20 k2=1.20 IDEAL_MASK")

ax[1].imshow(res2,cmap='gray',vmax=170)
ax[1].title.set_text("Inverse vmax=170 13° ITER D0=300 k1=0 k2=1.20 BUTTERWORTH_MASK")
plt.show()

In [ ]:
img = cv2.imread('pet.jpg',0)
runHPTests(img,D0=5,mask_type=IDEAL_MASK)

In [ ]:
img = cv2.imread('pet.jpg',0)
runHPTests(img,D0=300,mask_type=BUTTERWORTH_MASK)

In [ ]:
img = cv2.imread('pet.jpg',0)
runHPTests(img,D0=0.05,mask_type=GAUSSIAN_MASK)

# Question 6

**Given the DFT translation property:**

$f(x,y)e^{j2\pi(u_0x/M + v_0y/N)} \iff F(u-u_0,v-v_0)$ **and**

$f(x - x_0,y - y_0) \iff F(u,v)e^{j2\pi(ux_x/M + vy_0/N)}$ **,**

**show that** $f(x,y)(-1)^{x+y} \iff F(u-M/2,v-N/2)$.


As the book explains, a way to explain this equivalence is setting the following values $u_0 = M/2$ e $v_0 = N/2$ respectively


$$f(x,y)e^{j2\pi(u_0x/M + v_0y/N)} \iff F(u-u_0,v-v_0)$$

$$f(x,y)e^{j2\pi(M/2x/M + N/2y/N)} \iff F(u-M/2,v-N/2)$$

So the exponential term going to be $e^{j\pi x+y}$ that is the same of $(-1)^{x+y}$

$f(x,y)(-1)^{x+y} \iff F(u-M/2,v-N/2)$

If we assume the values x = 0 and y = 0 for the image coordinates, we obtain the center of DFT:

$f(x,y)(-1)^{x+y} \iff F(u-M/2,v-N/2)$

$f(0,0)(-1)^{0+0} \iff F(0-M/2,0-N/2)$

$f(0,0) \iff F(M/2,N/2)$

<br>

If we assume the values x = 5 and y = 5 for a 10x10 image, we will be in the center of the image, however, in the DFT we are on the top left (0,0) position:


$f(x,y)(-1)^{x+y} \iff F(u-M/2, v-N/2)$

$f(5,5)(-1)^{5+5} \iff F(5-10/2, 5-10/2)$

$f(5,5) \iff F(5-10/2, 5-10/2)$

$f(5,5) \iff F(0,0)$

That is, this is a way to reallocate the origin from matrix to your center. This procedure is called by *shift*, and is useful to analyze the spectrum of the image. When we calculate the DFT of one image the high values going to corners of spectrum, making it difficult to see the pattern, so we apply the shift, and we get a new image of spectrum where the values of corners going to center.

Despite the question not asking for a code, I always like to realize some practice to better understand, so the next explanations refer to my practical tests.

The next figure illustrates the movement of the origin of the matrix from the top left to the center.

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(16, 7)

def prepareAx(ax):
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    ax.invert_yaxis()
    ax.set_xticks([x for x in range(-10,11,5)])
    ax.set_yticks([x for x in range(-10,11,5)])
    ax.grid(True)

prepareAx(ax[0])
prepareAx(ax[1])
rect = patches.Rectangle((0, 0), 5, 5, linewidth=2, edgecolor='b', facecolor='none')
ax[0].add_patch(rect)

rect = patches.Rectangle((-2.5, -2.5), 5, 5, linewidth=2, edgecolor='b', facecolor='none')
ax[1].add_patch(rect)

plt.show()

# Practice

In the next code I implemented two functions according to the equation, the first *shift(dft)* it's refer to $F(u-M/2,v-N/2)$ and the second *shiftBefore(img)* is $f(x,y)(-1)^{x+y}$. The first should be applied to the DFT and the second to the image before calculate the DFT.

In [ ]:

def shift(dft):
    M, N = dft.shape
    new = np.zeros(dft.shape,dtype='complex')
    for x in range(M):
        for y in range(N):
            new[x,y] = dft[math.ceil(x-M/2),math.ceil(y-N/2)]
    return new

def shiftBefore(img,dtype='complex'):
    #new = np.zeros(img.shape,dtype=img.dtype)
    new = np.zeros(img.shape,dtype=dtype)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            new[x,y] = img[x, y] * ((-1)**(x+y))
    return new



Then I created one matrix to observer the action of *shift* more closely. Note that with the use of the first function, the colors that are in the corners going to the center of the image. It's like the image was divided in four parts, and these parts was reallocated.

The second function will invert the values of colors in certain positions, this affects the generated spectrum, causing one effect similar to the application of the first function that was applied directly on the spectrum.

In [ ]:
mat = np.asarray([[6,0,0,0,0,7],
                  [0,0,0,0,0,0],
                  [1,0,3,5,0,1],
                  [1,0,2,4,0,1],
                  [0,0,0,0,0,0],
                  [8,0,0,0,0,9]])
        
new = shift(mat).astype(int)

fig, ax = plt.subplots(2, 2)
fig.set_size_inches(10, 10)

ax[0,0].imshow(mat, cmap='Paired')
ax[0,0].title.set_text("Original")
ax[0,1].imshow(new, cmap='Paired')
ax[0,1].title.set_text("Shifted")


ax[1,0].imshow(mat, cmap='Paired')
ax[1,0].title.set_text("Original")
new = shiftBefore(mat).astype(int)
ax[1,1].imshow(new, cmap='Paired')
ax[1,1].title.set_text("Shift before")


plt.show()

In the code below I present one example of the comparison of *shift* implemented by me and the Numpy shift, despite the implementation of the Numpy shift be different, both got the same result. The last image was the inverse of spectrum, where was applied my implementation of the shift.

In [ ]:
fig, ax = plt.subplots(1, 4)
fig.set_size_inches(20, 6)

img = cv2.imread('pet.jpg',0)
#opencv shift
F1 = fp.fft2((img).astype(float))
F2 = fp.fftshift(F1)

#my shift
F2a = shift(F1)

ax[0].title.set_text("Spectrum Before Shift")
ax[0].imshow(spectrum(F1), cmap='gray')

ax[1].title.set_text("Spectrum OpenCV shift")
ax[1].imshow(spectrum(F2), cmap='gray')

ax[2].title.set_text("Spectrum My shift")
ax[2].imshow(spectrum(F2a), cmap='gray')

ax[3].title.set_text("Inverse of My shift")
ax[3].imshow(inverse(F2a), cmap='gray')

plt.show()


Next I demonstrate the effect of the second function, that is related to $f(x,y)^{x+y}$. The first is the original image, the second is the result of its application on the image, isn't possible identify anything, only a dotted texture, on the third image was applied $2 * log(img)$ to improve the visualization of the image, the fourth is the spectrum generated from this image and the fifth is the inverse of spectrum.

Obs: Note that the spectrum is slightly different, in visual terms when compared to the previous example, it's like "noiseless", but the result of reconstruction is the same in both.

In [ ]:
fig, ax = plt.subplots(1, 5)
fig.set_size_inches(20, 6)

img = cv2.imread('pet.jpg',0)

ax[0].title.set_text("Original Img")
ax[0].imshow(img, cmap='gray')


img = shiftBefore(img,dtype='float')
F1 = fp.fft2((img).astype(float))

#imagem do jeito que ficou
ax[1].title.set_text("Image after $(-1)^{x+y}$")
ax[1].imshow(img, cmap='gray')


#(20*np.log10( 0.1 + mag)).astype(int)
ax[2].title.set_text("Image after $2 * log((-1)^{x+y})$")
ax[2].imshow(2 * np.log(img).astype(int), cmap='gray')

ax[3].title.set_text("Spectrum of image")
ax[3].imshow(spectrum(F1), cmap='gray')


F1v = shiftBefore(fp.ifft2(F1).real).real

ax[4].title.set_text("Inverse of spectrum")
ax[4].imshow(F1v, cmap='gray')



plt.show()


# Question 7

<b>A professor of archeology doing research on currency exchange practices during the Roman Empire recently became aware that four Roman coins crucial to his research are listed in the holdings of the British Museum in London. Unfortunately, he was told after arriving there that the coins had been recently stolen. Further research on his part revealed that the museum keeps photographs of every item for which it is responsible. Unfortunately, the photos of the coins in question are blurred to the point where the date and other small markings are not readable. The cause of the blurring was the camera being out of focus when the pictures were taken. As an image processing expert and friend of the professor, you are asked as a favor to determine whether computer processing can be utilized to restore the images to the point where the professor can read the markings. You are told that the original camera used to take the photos is still available, as are other representative coins of the same era. Propose a step-by-step solution to this problem.</b>


To solve this problem, first we need to obtain the degradation function. I had a lot of difficult to understand this procedure, because, according to Gonzalez, et. al. (2008) explain, if we have the equipment that was used to capture the degraded photos, we can obtain a more accurate degradation function estimate, by capturing several images until we get one capture with most similar effect that one that we want to recover. Once that configuration was found, the next step is to determine the Point Spread Function (PSF). In this point I first was understood that we should, realize the capture of a small white point using the same configuration, but after my practical tests, I believe I got the translation wrong, and in this case it's as if the PSF was the mask that used to degrade the image.

But, once the degradation function is known, it's possible to apply the *Inverse Filter*, however, it's not indicated, because most of real cases, it couldn't have a good result if the degradation function values are low. The *Wiener filter* is most indicated.

The author presents the following equation to obtain the degradation function $H(u,v)$:

$H(u,v) = \dfrac{G(u,v)}{A}$

The $G(u,v)$ is the Fourier transform of the degraded image, and the $A$ is the strength of impulse, but I don't understand  how obtain the A value, and I didn't find examples. So I can't see the advantage of have the same equipment of original photo with this equation.

But, when the author mentions the subject **Estimation by Image Observation** he presents the following equation:

$H_s(u,v) = \dfrac{G_s(u,v)}{\hat{F}_s(u,v)}$

In this case the different between the first equation and the second is on the divisor, $\hat{F}_s(u,v)$, that is the Fourier transform of the same image with proper focus. The author explain that we can get part of image, preferentially  where exists some structure that we can enhance to improve the visualization, then we divide the Fourier transform of the degraded image to the Fourier of the enhanced image to obtain the degradation function.

With this procedure I got a better result, and the use of the original camera started to make sense. In this case the procedure is:
1. Take a photo of one coin where the degradation effect be close to the image that you want to restore.
2. Take one photo of the same coin with a proper configuration of focus.
3. Apply the equation above to get the degradation function.
3. Apply the following equation $F(u,v) = \dfrac{G(u,v)}{H(u,v)}$, to obtain the Fourier transform of the restored image.
5. Apply the inverse of Fourier to the result of the previous equation to obtain the restored image.

To get this conclusion the book reading wasn't enough, was necessary to realize practical experiments, next i presents the experiments that consist in:
- Apply computationally blur and restore the image (Success)
- Apply blur with smartphone camera and restore the image (Fail)
- Apply the Wiener Filter
    - In the degraded image artificially  (Success)
    - In the degraded image from phone (Fail)
- Apply the Richard Lucy
    - In the degraded image artificially  (Success)
    - In the degraded image from phone (Fail)
   


To this practice I adapted the code from: https://github.com/maponti/imageprocessing_course_icmc/blob/master/05b_restoration_deconvolution.ipynb


## Experiment 1 - Apply computationally blur and restore the image
### Apply blur on the images
First, I applied the Gaussian blur on the 5c and 25c coins until the year is illegible.

In [ ]:

def gaussian_filter(k=5, sigma=1.0):
    arx = np.arange((-k // 2) + 1.0, (k // 2) + 1.0)
    x, y = np.meshgrid(arx, arx)
    filt = np.exp(-(1/2) * (np.square(x) + np.square(y)) / np.square(sigma))
    return filt / np.sum(filt)


f = cv2.imread('coins/5_ok.jpg',0)
h = gaussian_filter(k=35, sigma=10.5)


#pega o h e deixa ele do tamanho da imagem onde será aplicado
#adiciona zeros nas bordas até ficar do tamanho desejado
# computing the number of padding on one side
a = int(f.shape[0]//2 - h.shape[0]//2)
h_pad = np.pad(h, (a,a-1), 'constant', constant_values=(0))


# computing the Fourier transforms
F = fftn(f)
H = fftn(h_pad)

plt.subplot(121)
plt.imshow(fftshift(np.log(np.abs(F)+1)), cmap="gray")
plt.subplot(122)
plt.imshow(fftshift(np.log(np.abs(H)+1)), cmap="gray")

# convolution - aplica o blur na imagem
G = np.multiply(F,H)

# Aplica o inverse
# Inverse Transform
# - we have to perform FFT shift before reconstructing the image in the space domain
g = fftshift(ifftn(G).real)
#g = cv2.blur(f,(25,25)) #Os dois métodos de aplicar blur geram o mesmo resultado

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)

ax[0].imshow(f, cmap="gray", vmin=0, vmax=255); 
ax[0].title.set_text("original image")

ax[1].imshow(g, cmap="gray", vmin=0, vmax=255);
ax[1].title.set_text("degraded/blurred image")

#Agora aplica o mesmo blur na de 25 cent
f25 = cv2.imread('coins/25_ok.jpg',0)
F25 = fftn(f25)
G25 = np.multiply(F25,H)
g25 = fftshift(ifftn(G25)).real

ax[2].imshow(g25, cmap="gray", vmin=0, vmax=255); 
ax[2].title.set_text("degraded/blurred image")


### Estimating the degradation function

From the 5c coin I can recover the degradation function, dividing the Fourier transform of the degraded image by the Fourier transform of the image with proper focus.

That is, the following equation:

$H_s(u,v) = \dfrac{G_s(u,v)}{\hat{F}_s(u,v)}$

But, instead of apply only one part of the image I applied the function on the whole image. Then it should be disregarded the $_s$ on the terms of above equation.


As soon as I get $H(u,v)$, I applied $\hat{F}(u,v) = \dfrac{G(u,v)}{H(u,v)}$ on the 5c coin that is degraded to create the Fourier transform of the image to be restored. Finally I apply the inverse to obtain the restored image.

With this, I can recover the 5c coin.

In [ ]:
#obtem a func de degradacao
H_obtido = np.divide(G,F)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G,H_obtido)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)

ax[0].title.set_text("degraded/blurred image")
ax[0].imshow(g, cmap="gray", vmin=0, vmax=255);

ax[1].title.set_text("restored")
ax[1].imshow(f_hat, cmap="gray"); plt.title("restored")

ax[2].title.set_text("Func degradação obtida")
ax[2].imshow(fftshift(np.log(np.abs(H)+1)), cmap="gray")
plt.show()

### Restoring the 25c coin

Now that the degradation function was obtained from the 5c coin, it's possible to restore the 25c coin.

In [ ]:
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G25,H_obtido)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g25, cmap="gray", vmin=0, vmax=255); plt.title("degraded/blurred image")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored")
plt.show()


### Experiment 2 - Apply blur with smartphone camera and restore the image

I tried to repeat the same procedure of experiment 1, however, instead of apply a blur computationally, I applied the blur at the moment that I was taking the photos. The next steps were the same.
- Take a photo of the 5c coin with blur and without blur.
- Take a photo of the 25c coin with blur.
- Restore the 25c coin from the degradation function obtained from the 5c coin.

I also tried to capture a white point focused and blurred to obtain the degradation function, but also without success, I only can restore the image that was used to create the degradation function.

Unfortunately  I didn't get a good result in this experiment, even the degradation function it's very different from expected. (Previous example)

In [ ]:
f = cv2.imread('coins/psfw_ok.jpg',0)
g = cv2.imread('coins/psfw_blur.jpg',0)

#f = cv2.imread('coins/5_ok.jpg',0)
#g = cv2.imread('coins/5_blur.jpg',0)


# computing the Fourier transforms
F = fftn(f)
G = fftn(g)
#obtem a func de degradacao
H = np.divide(G,F)
H_bk = H

#obtem a fourrier da imagem restaurada
F_hat = np.divide(G,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)

ax[0].title.set_text("degraded/blurred image")
ax[0].imshow(g, cmap="gray");

ax[1].title.set_text("Restored")
ax[1].imshow(f_hat, cmap="gray");

ax[2].title.set_text("Degradation function")
#ax[2].imshow(fftshift(np.log(np.abs(H)+1)), cmap="gray");
ax[2].imshow(spectrum(fftshift(H)), cmap="gray");
plt.show()


g5b = cv2.imread('coins/5_blur.jpg',0)
G5b = fftn(g5b)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G5b,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g5b, cmap="gray"); plt.title("degraded/blurred image")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored")
plt.show()



g25b = cv2.imread('coins/25_blur.jpg',0)
G25b = fftn(g25b)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G25b,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g25b, cmap="gray"); plt.title("degraded/blurred image")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored")
plt.show()



After some tests I was wondering if is possible to restore the same coin but with different images, so I tested:
- Capture 5c coin with proper focus (A)
- Capture 5c coin with blur (B)
- Capture another photo like the B
- Capture another photo like the B, but move the coin (D)
- Capture another photo like the B, but with the coin rotated (E)

The result is that I can't recover any photo, only that I used to get the degradation error.

In [ ]:

f = cv2.imread('coins/5c/5c_ok.jpg',0)
g = cv2.imread('coins/5c/5c_blur1.jpg',0)



# computing the Fourier transforms
F = fftn(f)
G = fftn(g)
#obtem a func de degradacao
H = np.divide(G,F)

#obtem a fourrier da imagem restaurada
F_hat = np.divide(G,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)

ax[0].title.set_text("degraded/blurred image (B)")
ax[0].imshow(g, cmap="gray");

ax[1].title.set_text("Restored (B)")
ax[1].imshow(f_hat, cmap="gray");

ax[2].title.set_text("Degradation function")
#ax[2].imshow(fftshift(np.log(np.abs(H)+1)), cmap="gray");
ax[2].imshow(spectrum(fftshift(H)), cmap="gray");
plt.show()


g5b = cv2.imread('coins/5c/5c_blur2.jpg',0)
G5b = fftn(g5b)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G5b,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g5b, cmap="gray"); plt.title("degraded/blurred image (C)")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored (C)")
plt.show()



g25b = cv2.imread('coins/5c/5c_blur3.jpg',0)
G25b = fftn(g25b)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G25b,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g25b, cmap="gray"); plt.title("degraded/blurred image (D)")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored (D)")
plt.show()



g25b = cv2.imread('coins/5c/5c_blur4.jpg',0)
G25b = fftn(g25b)
#obtem a fourrier da imagem restaurada
F_hat = np.divide(G25b,H)
#obtem a imagem restaurada
f_hat = ifftn(F_hat).real


plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(g25b, cmap="gray"); plt.title("degraded/blurred image (E)")
plt.subplot(122)
plt.imshow(f_hat, cmap="gray"); plt.title("restored (E)")
plt.show()



### Others methods to restore

I tried to apply others methods to restore the image using the same degradation function obtained  previously, also without success.

In [ ]:
h = fftshift(H_bk).real

res_img = rescale255(restoration.wiener(g25b, h, 0.1, clip=False))
res_img2 = rescale255(restoration.richardson_lucy(img_as_float(g25b), h, 1))

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(15, 6)

ax[0].imshow(g25b,cmap='gray',vmin=0,vmax=255)
ax[1].imshow(res_img,cmap='gray')
ax[2].imshow(res_img2,cmap='gray')
plt.show()

### Estimating the degradation function
I tried to apply the *Inverse filter* with many configurations, also without success.

In [ ]:
g = cv2.imread('coins/5_blur.jpg',0)
G = fftn(g)

for k in [5,15,35,55,105,255]:
    for sigma in np.arange(0.05,2.0,0.25):
        h = gaussian_filter(k=k, sigma=sigma)

        a = int(f.shape[0]//2 - h.shape[0]//2)
        if h.shape[0] % 2 == 0:
            h_pad = np.pad(h, (a,a), 'constant', constant_values=(0))
        else:
            h_pad = np.pad(h, (a,a-1), 'constant', constant_values=(0))
        H = fftn(h_pad)

        #F_hat = G/H
        F_hat = np.divide(G,H)

        #f_hat = ifftn(F_hat).real
        f_hat = fftshift(ifftn(F_hat).real)


        plt.figure(figsize=(12,5))
        plt.subplot(121)
        plt.imshow(g, cmap="gray", vmin=0, vmax=255); plt.title("degraded/blurred image")
        plt.subplot(122)
        plt.imshow(f_hat, cmap="gray", vmin=0, vmax=255); plt.title("restored k=%d sigma=%.2f "% (k,sigma))
        plt.show()


## Wiener method
### Applying the blur computationally and using the wiener filter

As I didn't get good results with the *Inverse filter*, I decided to test others methods, before I applied the blur computationally and restored the image thought the *Wiener filter*.

In [ ]:
img = cv2.imread('coins/25_ok.jpg',0)


blur = cv2.blur(img,(25,25))

bs = 25
psf = np.ones((bs, bs)) / (bs*bs)
res_img = rescale255(restoration.wiener(blur, psf, 0.1, clip=False))
#res_img = rescale255(restoration.richardson_lucy(blur, psf, 1))

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)

ax[0].imshow(blur,cmap='gray',vmin=0,vmax=255)
ax[1].imshow(res_img,cmap='gray',vmin=0,vmax=255)
plt.show()

### Restore the smartphone photo

Still unsuccessful to restore the smartphone photo.

In [ ]:
blur = cv2.imread('coins/25_blur.jpg',0)


def testFloat(blurSize):
    fig, ax = plt.subplots(1, 4)
    fig.set_size_inches(20, 6)
    i = 0
    #for f in (start=start, stop=stop, num=5):
    for f in [0.001,0.01,0.1,100]:
        psf = np.ones((blurSize, blurSize)) / (blurSize*blurSize)
        res_img = restoration.wiener(blur, psf, f, clip=False)

        ax[i].title.set_text("Mask Size %d and %.5f" % (blurSize,f))
        ax[i].imshow(res_img,cmap='gray',vmax=255)
        i += 1
    plt.show()
    
for blurSize in range(3,13,2):
    testFloat(blurSize=blurSize)

## Richardson lucy method

Also tried the *Richardson Lucy* method, this method can restore the computationally blur.


In [ ]:
img = cv2.imread('coins/25_ok.jpg',0)

blur = cv2.blur(img,(25,25))

bs = 25
psf = np.ones((bs, bs)) / (bs*bs)
res_img = rescale255(restoration.richardson_lucy(img_as_float(blur), psf, 35))

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)

ax[0].imshow(blur,cmap='gray',vmin=0,vmax=255)
ax[1].imshow(res_img,cmap='gray',vmin=0,vmax=255)
plt.show()

### Smarpthone photo restore with Richardson Lucy

As well as the Wienner method, this method can't restore the smartphone photo.

In [ ]:
#OBS ESTE MÉTODO É MAIS DEMORADO

blur = img_as_float(cv2.imread('coins/25_blur.jpg',0))

def testSizes():
    
    for it in range(30,81,20):
        fig, ax = plt.subplots(1, 4)
        fig.set_size_inches(20, 6)
        i = 0
        for blurSize in range(25,56,10):
            
            psf = np.ones((blurSize, blurSize)) / (blurSize*blurSize)
            res_img = rescale255(restoration.richardson_lucy(blur, psf, it))

            ax[i].title.set_text("Mask Size mask size=%d iteracoes=%d" % (blurSize,it))
            ax[i].imshow(res_img,cmap='gray')
            i += 1
        plt.show()

testSizes()

# Question 8

**8. Describe which problem you will try to solve in your end of course project, which are the Image Processing sub-areas involved and which techniques you intend to use.**

My course project is related to detection of malaria infection in blood smear. More precisely in segmentation and blood cells counting. The blood cells counting is important, because to determine the parasite density, it's required to have the most accurate possible count of blood cells. This project is based on the thesis of Oliveira (2019), so I pretend to realize the counting of blood cells using the same image bank of the original work and compare the results. The problem is that in the original work, some cells are so closely, that the author didn't separate the cells to count individually, instead, aa estimate was made based on area of the group of cells.
The original work used:
- HSV mask to detect the infection;
- Adaptative OTSU, to to binarize the image, separating the background and the blood cells.
- Hole filling by edge detection
- Connected components
- Erosion to separate components
In my work, I pretend to test others possibilities listed by Wu, Merchant and Castleman (2010) like Watershed and Boundary-Based Segmentation instead OTSU, but I will need to use algorithms of original work like, Connected components, hole filling and erosion.

The principal objective is to obtain a better blood cells segmentation.

The specific objectives is:
- Configure and run the code of original work to segment the images
- Implement others methods listed to segment images
- Compare segmentations methods in quality and performance


**Bibliography**
OLIVEIRA, Allisson Dantas de. MalariaApp: um sistema de baixo custo para diagnóstico de malária em lâminas de esfregaço sanguíneo usando dispositivos móveis. 2019.

WU, Qiang; MERCHANT, Fatima; CASTLEMAN, Kenneth (Ed.). Microscope image processing. Elsevier, 2010.